In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 51 kB/s 
     |████████████████████████████████| 199 kB 47.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=e4a45cda0c513e2c25ff60a9764c50c983377534599da349b34e32cf33d364df
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [4]:
spark

In [5]:
print(spark.catalog.listTables())

[]


In [7]:
!wget --continue https://raw.githubusercontent.com/GarvitArya/pyspark-demo/main/sample_books.json -O /tmp/sample_books.json

--2022-10-04 02:38:30--  https://raw.githubusercontent.com/GarvitArya/pyspark-demo/main/sample_books.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1565 (1.5K) [text/plain]
Saving to: ‘/tmp/sample_books.json’

/tmp/sample_books.j 100%[===================>]   1.53K  --.-KB/s    in 0s      

2022-10-04 02:38:30 (13.2 MB/s) - ‘/tmp/sample_books.json’ saved [1565/1565]



In [9]:
df = spark.read.json("/tmp/sample_books.json")
df.printSchema()

root
 |-- author: string (nullable = true)
 |-- edition: string (nullable = true)
 |-- price: double (nullable = true)
 |-- title: string (nullable = true)
 |-- year_written: long (nullable = true)



In [10]:
df.show(4,False)

+---------------+--------------+-----+----------------+------------+
|author         |edition       |price|title           |year_written|
+---------------+--------------+-----+----------------+------------+
|Austen, Jane   |Penguin       |18.2 |Northanger Abbey|1814        |
|Tolstoy, Leo   |Penguin       |12.7 |War and Peace   |1865        |
|Tolstoy, Leo   |Penguin       |13.5 |Anna Karenina   |1875        |
|Woolf, Virginia|Harcourt Brace|25.0 |Mrs. Dalloway   |1925        |
+---------------+--------------+-----+----------------+------------+
only showing top 4 rows



In [11]:
df.count()

13

In [12]:
df.select("title", "price", "year_written").show(5)

+----------------+-----+------------+
|           title|price|year_written|
+----------------+-----+------------+
|Northanger Abbey| 18.2|        1814|
|   War and Peace| 12.7|        1865|
|   Anna Karenina| 13.5|        1875|
|   Mrs. Dalloway| 25.0|        1925|
|       The Hours|12.35|        1999|
+----------------+-----+------------+
only showing top 5 rows



In [13]:
df_filtered = df.filter("year_written > 1950 AND price > 10 AND title IS NOT NULL")
df_filtered.select("title", "price", "year_written").show(50, False)

+-----------------------------+-----+------------+
|title                        |price|year_written|
+-----------------------------+-----+------------+
|The Hours                    |12.35|1999        |
|Harry Potter                 |19.95|2000        |
|One Hundred Years of Solitude|14.0 |1967        |
+-----------------------------+-----+------------+



In [15]:
df_filtered.select("title", "year_written").filter("title LIKE '%Harry Potter%'")\
.distinct().show(20, False)

+------------+------------+
|title       |year_written|
+------------+------------+
|Harry Potter|2000        |
+------------+------------+



In [18]:
from pyspark.sql.functions import max
maxValue = df_filtered.agg(max("price")).collect()[0][0]
print("maxValue: ",maxValue)
df_filtered.select("title","price").filter(df.price == maxValue).show(20, False)

maxValue:  19.95
+------------+-----+
|title       |price|
+------------+-----+
|Harry Potter|19.95|
+------------+-----+



In [19]:
print(spark.catalog.listTables())
df.createOrReplaceTempView("temp")
print(spark.catalog.listTables())

[]
[Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


In [23]:
query = "From temp SELECT * LIMIT 10"
books = spark.sql(query)
books.show()

+--------------------+--------------+-----+-------------------+------------+
|              author|       edition|price|              title|year_written|
+--------------------+--------------+-----+-------------------+------------+
|        Austen, Jane|       Penguin| 18.2|   Northanger Abbey|        1814|
|        Tolstoy, Leo|       Penguin| 12.7|      War and Peace|        1865|
|        Tolstoy, Leo|       Penguin| 13.5|      Anna Karenina|        1875|
|     Woolf, Virginia|Harcourt Brace| 25.0|      Mrs. Dalloway|        1925|
|Cunnningham, Michael|Harcourt Brace|12.35|          The Hours|        1999|
|         Twain, Mark|       Penguin| 5.76|   Huckleberry Finn|        1865|
|    Dickens, Charles|  Random House| 5.75|        Bleak House|        1870|
|         Twain, Mark|  Random House| 7.75|         Tom Sawyer|        1862|
|     Woolf, Virginia|       Penguin| 29.0|A Room of One's Own|        1922|
|       Rowling, J.K.|Harcourt Brace|19.95|       Harry Potter|        2000|